In [ ]:
#librerias
import pandas as pd
import json

In [33]:
"""
Aplicamos un etl a los datos descargados de la página de la salud de NYen formato json y los pasamos a formato CSV
"""

# Rutas de los archivos JSON y CSV
ruta_json = r'H:\git\proyecto grupal 2\Yelp-Gmaps-Proyecto-DS\data\raw\departamento salubridad NY\rows.json'
ruta_csv = r'H:\git\proyecto grupal 2\Yelp-Gmaps-Proyecto-DS\data\raw\departamento salubridad NY\rows.csv'

with open(ruta_json, 'r', encoding='utf-8') as f:
    data = json.load(f)

# Detectamos la estructura del JSON
if isinstance(data, dict) and 'data' in data:
    df = pd.DataFrame(data['data'])
elif isinstance(data, list):
    df = pd.DataFrame(data)
else:
    raise ValueError("❌ Estructura JSON no reconocida")

# Guardamos el DataFrame en un CSV sin cabecera
df.to_csv(ruta_csv, index=False, header=False, encoding='utf-8-sig')
print(f"✅ CSV generado desde JSON en: {ruta_csv}")

# Añadimos las cabeceras al archivo CSV
columnas_originales = [
    "row_id", "uuid", "col_1", "timestamp_1", "col_2", "timestamp_2", "col_3", "json_data",
    "borough", "restaurant_name", "location_type", "owner", "address", "inspection_id",
    "col_4", "compliance_status", "col_5", "inspection_date", "agency", "zip_code",
    "latitude", "longitude", "grade", "violation_points", "violation_code", 
    "bin", "bbl", "neighborhood"
]

df = pd.read_csv(ruta_csv, header=None, names=columnas_originales, low_memory=False, dtype=str)
df.to_csv(ruta_csv, index=False, encoding='utf-8-sig')
print(f"✅ Cabeceras aplicadas a: {ruta_csv}")

# Filtrarmos las columnas relevantes a usar
columnas_compatibles = {
    "restaurant_name": "nombre",
    "address": "direccion",
    "borough": "ciudad",
    "zip_code": "codigo_postal",
    "latitude": "latitud",
    "longitude": "longitud",
    "compliance_status": "estado_cumplimiento",
    "inspection_date": "fecha_inspeccion",
    "violation_points": "puntos_violacion",
    "violation_code": "codigo_violacion"
}

df_final = df[list(columnas_compatibles.keys())].rename(columns=columnas_compatibles)

# Normalizamos todos los datos string a minúsculas (sin tocar headers)
for col in df_final.columns:
    df_final[col] = df_final[col].astype(str).str.lower().str.strip()

# Cargamos el archivo de dimensión de nombres de restaurantes
ruta_dim = r'H:\git\proyecto grupal 2\Yelp-Gmaps-Proyecto-DS\data\processed\dim\dim_nombre_restaurante.csv'
dim_nombres = pd.read_csv(ruta_dim)

# Normalizamos 'nombre' en dimensión para que coincida
dim_nombres['nombre'] = dim_nombres['nombre'].astype(str).str.lower().str.strip()

# Eliminamos valores nulos y duplicados
df_final = df_final[df_final['nombre'].notna() & (df_final['nombre'].str.strip() != '')]
df_final = df_final.merge(dim_nombres, on='nombre', how='left')

# Ordenamos las columnas
orden_final = [
    'id_nombre', 'nombre', 'direccion', 'ciudad', 'codigo_postal',
    'latitud', 'longitud', 'estado_cumplimiento',
    'fecha_inspeccion', 'puntos_violacion', 'codigo_violacion'
]
df_final = df_final[orden_final]

# Guardamos el dataset limpio en un archivo CSV
ruta_salida = r'H:\git\proyecto grupal 2\Yelp-Gmaps-Proyecto-DS\data\processed\salubridad_merge_yelp.csv'
df_final.to_csv(ruta_salida, index=False, encoding='utf-8-sig')

print(f"✅ Dataset limpio, normalizado en minúsculas y unido guardado en:\n{ruta_salida}")



✅ CSV generado desde JSON en: H:\git\proyecto grupal 2\Yelp-Gmaps-Proyecto-DS\data\raw\departamento salubridad NY\rows.csv
✅ Cabeceras aplicadas a: H:\git\proyecto grupal 2\Yelp-Gmaps-Proyecto-DS\data\raw\departamento salubridad NY\rows.csv
✅ Dataset limpio, normalizado en minúsculas y unido guardado en:
H:\git\proyecto grupal 2\Yelp-Gmaps-Proyecto-DS\data\processed\salubridad_merge_yelp.csv


In [1]:
import pandas as pd
import hashlib

# Ruta del archivo CSV con datos de inspecciones de restaurantes
ruta_csv = r'H:\git\proyecto grupal 2\Yelp-Gmaps-Proyecto-DS\data\raw\departamento salubridad NY\DOHMH_New_York_City_Restaurant_Inspection_Results_20250413.csv'

# Cargar los datos
df = pd.read_csv(ruta_csv)

# 1. Renombrar las columnas a español
columnas_compatibles = {
    'CAMIS': 'ID_RESTAURANTE',
    'DBA': 'NOMBRE',
    'BORO': 'BOROUGH',
    'BUILDING': 'EDIFICIO',
    'STREET': 'CALLE',
    'ZIPCODE': 'CÓDIGO_POSTAL',
    'PHONE': 'TELÉFONO',
    'CUISINE DESCRIPTION': 'DESCRIPCIÓN_COCINA',
    'INSPECTION DATE': 'FECHA_INSPECCIÓN',
    'ACTION': 'ACCIÓN',
    'VIOLATION CODE': 'CÓDIGO_VIOLACIÓN',
    'VIOLATION DESCRIPTION': 'DESCRIPCIÓN_VIOLACIÓN',
    'CRITICAL FLAG': 'CRÍTICA',
    'SCORE': 'PUNTUACIÓN',
    'GRADE': 'CALIFICACIÓN',
    'GRADE DATE': 'FECHA_CALIFICACIÓN',
    'RECORD DATE': 'FECHA_REGISTRO',
    'INSPECTION TYPE': 'TIPO_INSPECCIÓN',
    'Latitude': 'LATITUD',
    'Longitude': 'LONGITUD',
    'Community Board': 'JUNTA_COMUNITARIA',
    'Council District': 'DISTRITO_CONCEJO',
    'Census Tract': 'SECCIÓN_CENSAL',
    'BIN': 'BIN',
    'BBL': 'BBL',
    'NTA': 'NTA',
    'Location Point1': 'PUNTO_UBICACIÓN'
}

df.rename(columns=columnas_compatibles, inplace=True)

# 2. Escanear los nombres de los restaurantes
nombres_restaurantes = df['NOMBRE'].dropna().drop_duplicates().reset_index(drop=True)

# 3. Cargar la dimensión actual (dim_nombre_restaurante)
ruta_dim = r'H:\git\proyecto grupal 2\Yelp-Gmaps-Proyecto-DS\data\processed\dim\dim_nombre_restaurante_limpia.csv'
dim_nombres = pd.read_csv(ruta_dim)

# 4. Verificar y agregar nombres que no estén en la dimensión
def generar_id(nombre):
    return "NOM" + hashlib.md5(nombre.encode()).hexdigest()[:6].upper()

# Escanear nombres y agregar nuevos si no están en la dimensión
nuevos_nombres = []
for nombre in nombres_restaurantes:
    if nombre not in dim_nombres['nombre'].values:
        id_nombre = generar_id(nombre)
        nuevos_nombres.append({'id_nombre': id_nombre, 'nombre': nombre})

# Convertir nuevos nombres a DataFrame
df_nuevos_nombres = pd.DataFrame(nuevos_nombres)

# 5. Concatenar la dimensión original con los nuevos nombres
dim_nombres_actualizada = pd.concat([dim_nombres, df_nuevos_nombres], ignore_index=True)

# 6. Guardar la nueva dimensión actualizada
dim_nombres_actualizada.to_csv(ruta_dim, index=False, encoding='utf-8-sig')

# 7. Guardar el DataFrame procesado en un nuevo archivo CSV
ruta_salida = r'H:\git\proyecto grupal 2\Yelp-Gmaps-Proyecto-DS\data\processed\salubridad_restaurantes_procesado.csv'
df.to_csv(ruta_salida, index=False, encoding='utf-8-sig')

print(f"✅ El DataFrame procesado se ha guardado en: {ruta_salida}")
print(f"✅ La dimensión actualizada se ha guardado en: {ruta_dim}")


✅ El DataFrame procesado se ha guardado en: H:\git\proyecto grupal 2\Yelp-Gmaps-Proyecto-DS\data\processed\salubridad_restaurantes_procesado.csv
✅ La dimensión actualizada se ha guardado en: H:\git\proyecto grupal 2\Yelp-Gmaps-Proyecto-DS\data\processed\dim\dim_nombre_restaurante_limpia.csv
